In [4]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.5.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools PyCall PyPlot Knet HDF5"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi


Installing Julia 1.5.2 on the current Colab Runtime...
2020-11-26 20:52:51 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.5/julia-1.5.2-linux-x86_64.tar.gz [105324048/105324048] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed VersionParsing ── v1.2.0
  Installed Artifacts ─────── v1.3.0
  Installed Parsers ───────── v1.0.12
  Installed ZMQ ───────────── v1.2.1
  Installed MbedTLS ───────── v1.0.3
  Installed IJulia ────────── v1.23.0
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed Conda ─────────── v1.5.0
  Installed JLLWrappers ───── v1.1.3
  Installed SoftGlobalScope ─ v1.1.0
  Installed JSON ──────────── v0.21.1
########################################################################

# How to run?

First know that this is a one-script notebook to run the entire project. 


1.   Run upper cell. (It install julia 1.5.2)
2.   Restart the notebook. (This allows you to use julia language)
3.   






In [3]:
versioninfo()

NameError: ignored